In [1]:
from pythonlib.tools.stroketools import *
from tools.preprocess import *
from sklearn.linear_model import LinearRegression
from pyvm.classes.videoclass import Videos
from tools.handtrack import HandTrack, getTrialsCameraFrametimes
from pyvm.utils.directories import get_metadata
from pythonlib.tools.expttools import load_yaml_config
from pyvm.globals import BASEDIR
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

In [14]:
def fit_regression_cam(HT, trange):
	"""
	Fits linear regression for this HT object and trial list
	PARAMS:
	HT, hand track object
	trange, range of trials
	RETURNS: 
	scikit linear regression object
	"""

	for trial in trange:

		dat, _ = HT.process_data_singletrial(trial, ploton=False, finger_raise_time = 0.0)

		if dat == {}:
			continue

	strokes_cam_all = []
	strokes_touch_all = []
	for strok_cam, strok_touch in zip(dat["strokes_cam"], dat["strokes_touch"]):
		strokes_cam_all.append(np.array(strok_cam))
		strokes_touch_all.append(np.array(strok_touch))

	assert (strokes_touch_all != {}), "No data for this expt"

	N = ["input_times"]
	for strok in strokes_cam_all:
		N.append(np.array([p[3] for p in strok]))

	touch_interp = strokesInterpolate2(strokes_touch_all, N)

	touch_interpz = []
	for strok in touch_interp:
		add_z = [[p[0],p[1],0,p[2]] for p in strok]
		touch_interpz.append(np.array(add_z))

	cam_one_list = np.array([])
	touch_one_list = np.array([])
	for strok_cam, strok_touch in zip(strokes_cam_all, touch_interpz):
		if len(cam_one_list) == 0 and len(touch_one_list) == 0:
			cam_one_list = np.array(strok_cam)
			touch_one_list = np.array(strok_touch)
		else:
			cam_one_list = np.concatenate((cam_one_list,strok_cam))
			touch_one_list = np.concatenate((touch_one_list,strok_touch))
			reg = LinearRegression().fit(cam_one_list, touch_one_list)
	return reg

In [15]:
#Get range of trials to analyze data from

ind1_ml2 = 1
config = load_yaml_config(f"/home/danhan/Documents/pipeline/metadata/221101_dirshapecolor3.yaml")
vid_inds = config["list_vidnums"][0]
trange = range(vid_inds[0],vid_inds[1])
ind1_vid = vid_inds[0]
print("Vid", trange)

animal = "Pancho"
date = "221101"
expt = "dirshapecolor3"
sess = "1"
sess_print = ""

fd = loadSingleDataQuick(animal, date, expt, sess)
HT = HandTrack(ind1_vid, ind1_ml2, fd, date=date, expt=expt)
HT.load_campy_data(1, sess=sess_print)
trials_no_ts_data = []



regression = fit_regression_cam(HT, trange)
HT.regressor = regression

# fd = loadSingleDataQuick("Pancho", "220317", "chunkbyshape4", 1)
# HT = HandTrack(4, 1, fd, 220317, "chunkbyshape4")

Vid range(2, 249)
/home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221101/221101_*_dirshapecolor3_Pancho_1.pkl
/home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221101/221101_*_dirshapecolor3_Pancho_1.pkl
-- loaded presaved data: /home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221101/221101_151509_dirshapecolor3_Pancho_1.pkl
TODO: Get accurate frametime after pass in frame extraction to Buttons
Searching using this string:
/data3/hand_track/Pancho/221101_dirshapecolor3/behavior/extracted_dlc_data/*camera*trial_4-*dat**
Found this many paths:
4
---
/data3/hand_track/Pancho/221101_dirshapecolor3/behavior/extracted_dlc_data/camera_bfs1_-trial_4-dat.pkl
---
/data3/hand_track/Pancho/221101_dirshapecolor3/behavior/extracted_dlc_data/camera_flea_-trial_4-dat.pkl
---
/data3/hand_track/Pancho/221101_dirshapecolor3/behavior/extracted_dlc_data/camera_fly1_-trial_4-dat.pkl
---
/data3/hand_track/Pancho/221101_dirshapecolor3/behavi

AssertionError: op1=635, op2=708